In [1]:
%matplotlib qt

In [2]:
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
folder = 'results/'

# Get the files in the folder
import os
files = os.listdir(folder)
files_vr = [f for f in files if f.endswith('particles.dat')]
files_vr.sort()
files_sol = [f for f in files if f.endswith('pm_output.dat')]
files_sol.sort()

In [4]:
import multiprocessing as mp
ALL_XS = []
ALL_YS = []
ALL_ZS = []
ALL_UXS = []
ALL_UYS = []
ALL_UZS = []
ALL_QXS = []
ALL_QYS = []
ALL_QZS = []

def process_file(filename):
    """Process a single file and return the data arrays."""
    data = {
        "XS": [],
        "YS": [],
        "ZS": [],
        "UXS": [],
        "UYS": [],
        "UZS": [],
        "QXS": [],
        "QYS": [],
        "QZS": []
    }

    with open(folder + filename) as file:
        lines = file.readlines()
        vars = lines[0].split()

        # From the file name get the NTIME
        ntime = int(filename[:5])

        # Initialize lists for each variable
        xs, ys, zs, uxs, uys, uzs, qxs, qys, qzs = [], [], [], [], [], [], [], [], []

        for l in lines[2:]:
            l = l.split()
            try:
                id = int(l[0])
                x = float(l[1])
                y = float(l[2])
                z = float(l[3])
                u = float(l[4])
                v = float(l[5])
                w = float(l[6])
                vort_x = float(l[7])
                vort_y = float(l[8])
                vort_z = float(l[9])
            except ValueError:
                print('Error in line: ', l)
                continue

            xs.append(x)
            ys.append(y)
            zs.append(z)
            uxs.append(u)
            uys.append(v)
            uzs.append(w)
            qxs.append(vort_x)
            qys.append(vort_y)
            qzs.append(vort_z)

        # Convert lists to NumPy arrays
        data["XS"] = np.array(xs)
        data["YS"] = np.array(ys)
        data["ZS"] = np.array(zs)
        data["UXS"] = np.array(uxs)
        data["UYS"] = np.array(uys)
        data["UZS"] = np.array(uzs)
        data["QXS"] = np.array(qxs)
        data["QYS"] = np.array(qys)
        data["QZS"] = np.array(qzs)

    return data

# Process files in parallel
with mp.Pool() as pool:
    results = pool.map(process_file, files_vr)

# Collect results
for result in results:
    ALL_XS.append(result["XS"])
    ALL_YS.append(result["YS"])
    ALL_ZS.append(result["ZS"])
    ALL_UXS.append(result["UXS"])
    ALL_UYS.append(result["UYS"])
    ALL_UZS.append(result["UZS"])
    ALL_QXS.append(result["QXS"])
    ALL_QYS.append(result["QYS"])
    ALL_QZS.append(result["QZS"])


In [5]:
ALL_PM_XS = []
ALL_PM_YS = []
ALL_PM_ZS = []
ALL_PM_UXS = []
ALL_PM_UYS = []
ALL_PM_UZS = []
ALL_PM_QXS = []
ALL_PM_QYS = []
ALL_PM_QZS = []
ALL_PM_PSIX = []
ALL_PM_PSIY = []
ALL_PM_PSIZ = []
ALL_PM_DEFORMX = []
ALL_PM_DEFORMY = []
ALL_PM_DEFORMZ = []

import pandas as pd

def process_file_pm(f):
    with open(folder + f) as file:
        lines = file.readlines()
        vars = lines[0].split()
        vars = vars[2:]
        vars = [v.replace('"','') for v in vars]
        
        # From the file name get the NTIME
        ntime = int(f[:5])
        sizes = [i for i in lines[1].split()]
        IS = int(sizes[1][2:])
        JS = int(sizes[2][2:])
        KS = int(sizes[3][2:])
        
    df = pd.read_csv(folder + f,sep=r'\s+',header=None, skiprows=3) 
    # Make the vars the column names
    df.columns = vars
    return df, IS, JS, KS

# Process files in parallel
with mp.Pool() as pool:
    results = pool.map(process_file_pm, files_sol)

# Collect results
for result in results:
    df, IS, JS, KS = result

    XS = df['X'].values.reshape(KS, JS, IS, order='C')
    YS = df['Y'].values.reshape(KS, JS, IS, order='C')
    ZS = df['Z'].values.reshape(KS, JS, IS, order='C')
    UXS = df['U'].values.reshape(KS, JS, IS, order='C')
    UYS = df['V'].values.reshape(KS, JS, IS, order='C')
    UZS = df['W'].values.reshape(KS, JS, IS, order='C')
    QXS = df['VORTX'].values.reshape(KS, JS, IS, order='C')
    QYS = df['VORTY'].values.reshape(KS, JS, IS, order='C')
    QZS = df['VORTZ'].values.reshape(KS, JS, IS, order='C')
    PSIX = df['PSI1'].values.reshape(KS, JS, IS, order='C')
    PSIY = df['PSI2'].values.reshape(KS, JS, IS, order='C')
    PSIZ = df['PSI3'].values.reshape(KS, JS, IS, order='C')
    DEFORMX = df['DEFORMX'].values.reshape(KS, JS, IS, order='C')
    DEFORMY = df['DEFORMY'].values.reshape(KS, JS, IS, order='C')
    DEFORMZ = df['DEFORMZ'].values.reshape(KS, JS, IS, order='C')

    ALL_PM_XS.append(XS)
    ALL_PM_YS.append(YS)
    ALL_PM_ZS.append(ZS)
    ALL_PM_UXS.append(UXS)
    ALL_PM_UYS.append(UYS)
    ALL_PM_UZS.append(UZS)
    ALL_PM_QXS.append(QXS)
    ALL_PM_QYS.append(QYS)
    ALL_PM_QZS.append(QZS)
    ALL_PM_PSIX.append(PSIX)
    ALL_PM_PSIY.append(PSIY)
    ALL_PM_PSIZ.append(PSIZ)
    ALL_PM_DEFORMX.append(DEFORMX)
    ALL_PM_DEFORMY.append(DEFORMY)
    ALL_PM_DEFORMZ.append(DEFORMZ)
    


/tmp/ipykernel_170167/3250573426.py:33: DtypeWarning: Columns (6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(folder + f,sep=r'\s+',header=None, skiprows=3)


In [6]:
print(len(ALL_XS))
print(len(ALL_PM_XS))
x_min = np.min([np.min(x) for x in ALL_XS])
x_max = np.max([np.max(x) for x in ALL_XS])
y_min = np.min([np.min(y) for y in ALL_YS])
y_max = np.max([np.max(y) for y in ALL_YS])
z_min = np.min([np.min(z) for z in ALL_ZS])
z_max = np.max([np.max(z) for z in ALL_ZS])
print(x_min, x_max)
print(y_min, y_max)
print(z_min, z_max)

101
101
-1.185502 1.916142
-1.272526 1.817727
-1.014044 3.540122


In [7]:
fig = plt.figure(figsize=(10, 5))

time = 1
# Get the shapes of the arrays
shape = ALL_PM_XS[time].shape

# Calculate the middle indices for each dimension
mid_x = shape[2] // 2
mid_y = shape[1] // 2
mid_z = shape[0] // 2

# Create proper slices (remember, the order is Z, Y, X in the array)
slice_xy = (mid_z, slice(None), slice(None))  # X-Y plane at middle Z
slice_xz = (slice(None), mid_y, slice(None))  # X-Z plane at middle Y

ax_vorticity1 = fig.add_subplot(121)
ax_vorticity2 = fig.add_subplot(122)

# Calculate vorticity magnitude
vorticity = np.sqrt(ALL_PM_QXS[time]**2 + ALL_PM_QYS[time]**2 + ALL_PM_QZS[time]**2)

# X-Y plane contour
x_xy = ALL_PM_XS[time][slice_xy].T  # Transpose to get correct orientation
y_xy = ALL_PM_YS[time][slice_xy].T
vorticity_xy = vorticity[slice_xy].T

contour1 = ax_vorticity1.contourf(x_xy, y_xy, vorticity_xy, cmap='viridis')
fig.colorbar(contour1, ax=ax_vorticity1)

# X-Z plane contour
x_xz = ALL_PM_XS[time][slice_xz].T
z_xz = ALL_PM_ZS[time][slice_xz].T
vorticity_xz = vorticity[slice_xz].T

contour2 = ax_vorticity2.contourf(x_xz, z_xz, vorticity_xz, cmap='viridis')
fig.colorbar(contour2, ax=ax_vorticity2)

# Set labels for vorticity plots
z_value = ALL_PM_ZS[time][mid_z, 0, 0]
y_value = ALL_PM_YS[time][0, mid_y, 0]

ax_vorticity1.set_title(f'Vorticity (X-Y plane) Z={z_value:.2f}')
ax_vorticity1.set_xlabel('X')
ax_vorticity1.set_ylabel('Y')

ax_vorticity2.set_title(f'Vorticity (X-Z plane) Y={y_value:.2f}')
ax_vorticity2.set_xlabel('X')
ax_vorticity2.set_ylabel('Z')

plt.tight_layout()
plt.show()

In [8]:
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.axes import Axes

time = 1

# Create the figure
# ax: 3D axis for particles
# ax2: 3D axis for mesh points
fig = plt.figure(figsize=(10, 10))
ax_particles: Axes3D = fig.add_subplot(121, projection='3d')
ax_grid_pm: Axes3D = fig.add_subplot(122, projection='3d')


# Set labels
# Set views
for ax in [ax_particles, ax_grid_pm]:
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=30)
    ax.set_aspect('equal')
    ax.set_xlim(x_min, x_max)



# Scatter plots for particles and mesh points
particle_velocities = np.sqrt(ALL_UXS[time]**2 + ALL_UYS[time]**2 + ALL_UZS[time]**2)
particles = ax_particles.scatter(ALL_XS[time], ALL_YS[time], ALL_ZS[time], 
                                c=particle_velocities, s=10, marker='o')
# Colorbar
fig.colorbar(particles, ax=ax_particles)

mesh_velocities = np.sqrt(ALL_PM_UXS[time]**2 + ALL_PM_UYS[time]**2 + ALL_PM_UZS[time]**2)
mesh_points = ax_grid_pm.scatter(ALL_PM_XS[time], ALL_PM_YS[time], ALL_PM_ZS[time],
                                c=mesh_velocities, s=0.5, marker='x')
# Colorbar
fig.colorbar(mesh_points, ax=ax_grid_pm)

# Set labels for particle plots
ax_particles.set_title('Particles')
ax_particles.set_xlabel('X')
ax_particles.set_ylabel('Y')
ax_particles.set_zlabel('Z')

ax_grid_pm.set_title('Mesh Points')
ax_grid_pm.set_xlabel('X')
ax_grid_pm.set_ylabel('Y')
ax_grid_pm.set_zlabel('Z')

Text(0.5, 0, 'Z')